<h1 style="color:blue;">  Scenario 11  </h1>

- C2.S11.Py01	Import AllReviews and create text data	
- C2.S11.Py02	Create Bigrams with Bag of Words and Count Vectorization with Negative True Reviews	
- C2.S11.Py03	Extra cleaning for the data based on the removal of numbers	
- C2.S11.Py04	Re-run Count Vectorization for Bigrams, Trigrams, and Quadgrams	
- C2.S11.Py05	Clean and run count vectorization for positive true and compare to negative true reviews	
- C2.S11.Py06	Diving into common n-grams	




In [3]:
#Code Block 1

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)

import matplotlib.pyplot as plt


# New trick to widen the screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))


import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = FutureWarning)


from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/1704621519.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Business Objective
- Identify key words specifically for negative comments that are true
- Idenitfy categories of comments to get a better understanding of the themes for negative true comments
- Conduct the same analysis for positive true comments

### Technical Objective
- Use bag of words for finding key words for negative true
- Use a vectorized approach (CountVectorization) for finding key words

<h2 style="color:blue;"> Import all reviews and create text data    </h2>

In [5]:
#Code Block 2

url = 'Scenario11_Data/Scenario11.csv'
df =  pd.read_csv(url, index_col=0) 

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1600 entries, 0 to 1599
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Reviews     1600 non-null   object
 1   Type        1600 non-null   object
 2   Status      1600 non-null   object
 3   TypeStatus  1600 non-null   object
dtypes: object(4)
memory usage: 62.5+ KB


In [7]:
#Code Block 3

df.head()

,Reviews,Type,Status,TypeStatus
0,"My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one...",NEG,TRUE,Negative True
1,The historic feel of the hotel really had a strong influence on my choice to book this hotel. Un...,NEG,TRUE,Negative True
2,I haven't actually stayed at this hotel- yet- but am already disappointed in their customer serv...,NEG,TRUE,Negative True
3,"I was very much looking forward to our stay at this hotel, after hearing all the good things abo...",NEG,TRUE,Negative True
4,The hotel is almost always very helpful. This stay caused me to re-think if I like the Palmer Ho...,NEG,TRUE,Negative True


In [9]:
#Code Block 4

df_neg = df[df['TypeStatus']=='Negative True']
df_neg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Reviews     400 non-null    object
 1   Type        400 non-null    object
 2   Status      400 non-null    object
 3   TypeStatus  400 non-null    object
dtypes: object(4)
memory usage: 15.6+ KB


In [11]:
#Code Block 5

df_pos = df[df['TypeStatus']=='Positive True']
df_pos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 800 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Reviews     400 non-null    object
 1   Type        400 non-null    object
 2   Status      400 non-null    object
 3   TypeStatus  400 non-null    object
dtypes: object(4)
memory usage: 15.6+ KB


In [13]:
#Code Block 6

df_neg.head()

,Reviews,Type,Status,TypeStatus
0,"My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one...",NEG,TRUE,Negative True
1,The historic feel of the hotel really had a strong influence on my choice to book this hotel. Un...,NEG,TRUE,Negative True
2,I haven't actually stayed at this hotel- yet- but am already disappointed in their customer serv...,NEG,TRUE,Negative True
3,"I was very much looking forward to our stay at this hotel, after hearing all the good things abo...",NEG,TRUE,Negative True
4,The hotel is almost always very helpful. This stay caused me to re-think if I like the Palmer Ho...,NEG,TRUE,Negative True


In [15]:
#Code Block 7

import re
import string
import nltk
pd.set_option('display.max_colwidth', 100) #Allows you to view up to 100 characters

#pd.set_option('display.max_colwidth', -1) #Allow you to view all text in the review

### Filter Data

#### Replace all numbers for ''

In [18]:
#Code Block 9

df_neg['Reviews'] = df_neg['Reviews'].str.replace('\d+', '')

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/780464151.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_neg['Reviews'] = df_neg['Reviews'].str.replace('\d+', '')
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/780464151.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['Reviews'] = df_neg['Reviews'].str.replace('\d+', '')


In [20]:
#Code Block 10

from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.util import ngrams
from collections import Counter

stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/donovanmanogue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#Code Block 11

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([wordnet_lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return text

df_neg['cleaned_Reviews'] = df_neg['Reviews'].apply(lambda x: clean_text(x))
df_neg.head()

<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\W'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/1833522329.py:5: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', text)
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/1833522329.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['cleaned_Reviews'] = df_neg['Reviews'].apply(lambda x: clean_text(x))


,Reviews,Type,Status,TypeStatus,cleaned_Reviews
0,"My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one...",NEG,TRUE,Negative True,wife spent long weekend hardrock chicago hotel although one normally write long email felt neces...
1,The historic feel of the hotel really had a strong influence on my choice to book this hotel. Un...,NEG,TRUE,Negative True,historic feel hotel really strong influence choice book hotel unfortunately ambiance hotel could...
2,I haven't actually stayed at this hotel- yet- but am already disappointed in their customer serv...,NEG,TRUE,Negative True,havent actually stayed hotel yet already disappointed customer service visit chicago quite often...
3,"I was very much looking forward to our stay at this hotel, after hearing all the good things abo...",NEG,TRUE,Negative True,much looking forward stay hotel hearing good thing unfortunately underwhelming experience say le...
4,The hotel is almost always very helpful. This stay caused me to re-think if I like the Palmer Ho...,NEG,TRUE,Negative True,hotel almost always helpful stay caused rethink like palmer house dirty cover bed also smallest ...


<h2 style="color:blue;"> Create a bag of words for Bigrams with Negative True  </h2>

### Bigrams with Bag of Words

In [46]:
#Code Block 12

neg_text = df_neg['cleaned_Reviews']

In [48]:
#Code Block 13

neg_text = neg_text.to_string()
neg_text = nltk.word_tokenize(neg_text)
neg_text

['0',
 'wife',
 'spent',
 'long',
 'weekend',
 'hardrock',
 'chicago',
 'hotel',
 'although',
 'one',
 'normally',
 'write',
 'long',
 'email',
 'felt',
 'neces',
 '...',
 '1',
 'historic',
 'feel',
 'hotel',
 'really',
 'strong',
 'influence',
 'choice',
 'book',
 'hotel',
 'unfortunately',
 'ambiance',
 'hotel',
 'could',
 '...',
 '2',
 'havent',
 'actually',
 'stayed',
 'hotel',
 'yet',
 'already',
 'disappointed',
 'customer',
 'service',
 'visit',
 'chicago',
 'quite',
 'often',
 '...',
 '3',
 'much',
 'looking',
 'forward',
 'stay',
 'hotel',
 'hearing',
 'good',
 'thing',
 'unfortunately',
 'underwhelming',
 'experience',
 'say',
 'le',
 '...',
 '4',
 'hotel',
 'almost',
 'always',
 'helpful',
 'stay',
 'caused',
 'rethink',
 'like',
 'palmer',
 'house',
 'dirty',
 'cover',
 'bed',
 'also',
 'smallest',
 '...',
 '5',
 'swissotel',
 'totally',
 'understaffed',
 'lacking',
 'charm',
 'atmosphere',
 'clean',
 'comfortable',
 'great',
 'view',
 'wanted',
 'chec',
 '...',
 '6',
 'ima

In [50]:
#Code Block 14

bigrams_neg = ngrams(neg_text, 2)
Counter(bigrams_neg).most_common(30)

[(('stayed', 'hotel'), 15),
 (('booked', 'hotel'), 14),
 (('front', 'desk'), 13),
 (('room', 'small'), 11),
 (('stay', 'hotel'), 10),
 (('good', 'location'), 10),
 (('hotel', 'room'), 10),
 (('one', 'night'), 9),
 (('hotel', '...'), 8),
 (('room', 'service'), 8),
 (('chicago', 'hotel'), 7),
 (('great', 'location'), 7),
 (('hotel', 'chicago'), 7),
 (('l', '...'), 7),
 (('2', 'night'), 7),
 (('hard', 'rock'), 6),
 (('s', '...'), 6),
 (('king', 'size'), 6),
 (('size', 'bed'), 6),
 (('hotel', 'monaco'), 6),
 (('customer', 'service'), 5),
 (('palmer', 'house'), 5),
 (('get', 'room'), 5),
 (('requested', 'room'), 5),
 (('room', '...'), 5),
 (('hotel', 'nice'), 5),
 (('gave', 'u'), 5),
 (('downtown', 'chicago'), 5),
 (('c', '...'), 5),
 (('hotel', 'allegro'), 5)]

### Count Vectorization for Bigrams with Negative True 

In [53]:
#Code Block 15

from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2,2))
X_counts = ngram_vect.fit_transform(df_neg['cleaned_Reviews'])
print(X_counts.shape)
print(ngram_vect.get_feature_names())

(400, 28271)


AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [55]:
#Code Block 16

X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()
X_counts_df.tail()

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

In [57]:
#Code Block 17

comments_sum = X_counts_df.sum()
comments_sum = comments_sum.sort_values(ascending=False)
comments_sum.head(30)

9994     116
20883     64
23531     35
6303      30
20901     29
16254     24
12152     24
28039     24
6984      22
23365     22
10924     22
28021     20
6849      20
8399      20
18198     19
15906     19
6208      19
23675     18
11312     18
10817     18
17277     18
9382      17
3768      17
16315     17
10548     17
14526     16
28000     16
12186     16
3166      16
22441     15
dtype: int64

<h2 style="color:blue;"> Extra cleaning for the data based on the removal of numbers   </h2>

### Why th floor?

- We removed the numbers, so it changed the floor numbers, such as '11th floor' to 'th floor'.
- Also need to look for 'st' for 1st and 'nd' for 2nd.

In [75]:
#Code Block 18

pd.set_option('display.max_colwidth', 1)
df_neg[df_neg['cleaned_Reviews'].str.contains("th floor")]

,Reviews,Type,Status,TypeStatus,cleaned_Reviews
1,"The historic feel of the hotel really had a strong influence on my choice to book this hotel. Unfortunately, the ambiance of the hotel could not make up for the rest of its shortcomings. The front desk staff was rude, the doormen were usually busy chatting or playing on their cell phones instead of greeting us or opening the door. There are three elevators but good luck with them all working. They are very very small and when only one is up and running...you end up waiting a long time for one. Not fun when you are on the 12th floor. The room was TINY and the air conditioning did not work at all...the room felt damp and miserable. We put the thermostat as low as it would let us (65) and the room never went below 78. (and it was 107 outside the whole trip!!!). There really are no amenities inside the hotel other than an ice machine and a slightly overpriced restaurant. Internet is not free either... Overall, I would never stay here again.",NEG,TRUE,Negative True,historic feel hotel really strong influence choice book hotel unfortunately ambiance hotel could make rest shortcoming front desk staff rude doorman usually busy chatting playing cell phone instead greeting u opening door three elevator good luck working small one runningyou end waiting long time one fun 12th floor room tiny air conditioning work allthe room felt damp miserable put thermostat low would let u 65 room never went 78 107 outside whole trip really amenity inside hotel ice machine slightly overpriced restaurant internet free either overall would never stay
4,"The hotel is almost always very helpful. This stay caused me to re-think if I like the Palmer House. I had dirty covers for the bed. It was also the smallest room in the hotel I am sure. My husband and I usually come 8 times each year. He is there doing Business and CTA work. We are not pleased with the size of the room at all now. Rooms have always had two chairs, desk, and usually 22 by 25 feet. Not this room on the 11th floor. It was only 15 X 17 approx. The service men at the doors have always said a welcome but not this time. In the past we had a bell man steal a $300. pair of sun glasses and the hotel did nothing about our loss financially.",NEG,TRUE,Negative True,hotel almost always helpful stay caused rethink like palmer house dirty cover bed also smallest room hotel sure husband usually come 8 time year business cta work pleased size room room always two chair desk usually 22 25 foot room 11th floor 15 x 17 approx service men door always said welcome time past bell man steal 300 pair sun glass hotel nothing loss financially
13,"Overview: --Overrated hotel in a premium location. The Great: --Location - walking distance or cheap cab ride to a ton of great restaurants, bars, and sights. The Good: --Inviting lobby, pleasant staff though a few were a bit snobby, in particular the night deskman. --Nice bathroom, comfortable bed and desk chair. The Mediocre: --Room (excluding bathroom) was underwhelming in design. --No kitchen or bar though they have limited room service from Harry Caray's across the street. --Free cocktails 5-7pm in ""lounge"" is right idea with wrong execution; tiny little room packed with people and a line extending out the door. --Hot water (or lack thereof) - almost every morning my shower water temperature would waver between comfortably hot to lukewarm. Pressure is pretty low. --View - mine wasn't great but apparently some of the rooms are horrible. The Bad: --Free wi-fi / ethernet has inconstant speeds and required multiple logins every day, even when my computer wasn't moved - really aggravating. Same deal for my iPhone. I also got numerous vulnerability threat warnings from my Norton Anti-virus that makes me wonder how secure it is. --No blackout curtains! I had a corner room with a huge wrap-around window and awoke to bright sunlight every morning (not to mention a near total lack of privacy at night). Never had this issue in

In [77]:
#Code Block 19

df_neg[df_neg['cleaned_Reviews'].str.contains("st floor")]

,Reviews,Type,Status,TypeStatus,cleaned_Reviews
152,"My husband and I stayed overnight at the Affinia. Prior reviews led us to believe this was a quality hotel; not true. Since it was a very special occasion, and also because we were only staying overnight, we didn't let the shabbiness and poor service of the hotel ruin our weekend. However, this would have qualified as one of the poorest hotel experiences. I've ever had. First, the smells: The linens and pillows on our bed smelled moldy and mildewy. The odors were not mild, and I found it pretty repulsive. I couldn't wait to jump out of bed in the morning, to get away from the odors. Hard to sleep and certainly didn't want to be in the bed any longer than absolutely necessary. I didn't discover how disgusting the bed smelled until around midnight when we got into bed. At that point, I chose not to contact housekeeping because (a) I was exhausted, and (b) I didn't want to upset my husband, who had fallen immediately to sleep after a long day of sightseeing. Also, the bathroom smelled like vinegar. Perhaps this is the cleaning agent used by Affinia, but the odor should have dissipated over the course of our stay and it never did. Incidentally, the bathroom floor tiles and wallpaper looked like they had seen better days; not a well-maintained property. Second, the elevators: If you stay above the first few floors, there are only two elevators that run from ground level to the rooftop c-view bar. With so many floors to service, and so many people going up to the c-view bar on a saturday, these elevators took forever to come, and were often full they did. Annoying. Also, on one occasion, there were several bottles (water and soda) left in the elevator; clearly, the maintenance crew doesn't inspect and clean the common areas frequently. Finally, and perhaps the most disappointing was the service when we arrived. First, when we booked, we asked for as quiet a room as possible. The hotel is located near Northwestern Memorial Hospital. But, its proximity to Michigan Ave., and the lack of accomodations elsewhere (due to the Transformers movie filming and a big international convention taking place the same week-end), led us to stay here anyway. At check-in, the clerk noted our request, and said our room was quiet room since it was not ""next to the elevators."" When we arrived at our room, we discovered that we were across from the elevators. Obviously, she was technically correct that we weren't next to the elevators, but it was still so close (15-20 feet) that we could hear the elevator noise. Also, since it was a special occasion, we asked if we could upgrade. The disingenuous, snarky, condescending clerk told us that checking in early was an upgrade. Clearly, the woman has not travelled enough to know that in many quality hotels, this is a matter of routine, not exception. Pretty awful experience all the way around. Other reviewers must have very low expectations. Can't explain any of the good reviews otherwise.",NEG,TRUE,Negative True,husband stayed overnight affinia prior review led u believe quality hotel true since special occasion also staying overnight didnt let shabbiness poor service hotel ruin weekend however would qualified one poorest hotel experience ive ever first smell linen pillow bed smelled moldy mildewy odor mild found pretty repulsive couldnt wait jump bed morning get away odor hard sleep certainly didnt want bed longer absolutely necessary didnt discover disgusting bed smelled around midnight got bed point chose contact housekeeping exhausted b didnt want upset husband fallen immediately sleep long day sightseeing also bathroom smelled like vinegar perhaps cleaning agent used affinia odor dissipated course stay never incidentally bathroom floor tile wallpaper looked like seen better day wellmaintained property second elevator stay first floor two elevator run ground level rooftop cview bar many floor service many people going cview bar saturday elevator took for

In [79]:
#Code Block 20

df_neg[df_neg['cleaned_Reviews'].str.contains("nd floor")]

,Reviews,Type,Status,TypeStatus,cleaned_Reviews
97,"My sister and I stayed on the 22nd floor, there was a refrigerator in the hallway for the first three nights, beat up and old dorm style. It was not until we told the concierge that we were going to post the picture we took of it on Tripadvisor did the thing get removed. Breakfast dishes left outside rooms will stay on the floor for a day or two routinely. All night there are chimes that make noise, that are supposed to be comformting or relaxing and rather they are just annoying. There is no wifi - no matter what they tell you, I paid for the wifi, and still we were unable to get on. Lastly, my sister and I split the bill, I had already paid for the first night in form of a deposit and they gave her the credit. Room cost with city taxes etc. was 313.69 a night. I know the Comfort Inn just a few blocks away is significantly cheaper! So while the View was wonderful, it is NOT worth the money.",NEG,TRUE,Negative True,sister stayed 22nd floor refrigerator hallway first three night beat old dorm style told concierge going post picture took tripadvisor thing get removed breakfast dish left outside room stay floor day two routinely night chime make noise supposed comformting relaxing rather annoying wifi matter tell paid wifi still unable get lastly sister split bill already paid first night form deposit gave credit room cost city tax etc 31369 night know comfort inn block away significantly cheaper view wonderful worth money
139,"We traveled to Chicago to visit with our daughter who lives in the city. To our suprise what once was a lovely older hotel is now under major renovations. Gone is the lobby and gusts are required to walk down to the basement to get the one... elevator that is working to the second floor to register. Our first room was recently renovated.....stark white walls, no amenities at all. Guests had to enter the room and find their way to the bed to turn on the light to the room. Modern furniture includes a bed that is close to the groud, a cold blck slab functioning a a desh and metal chair. No fride, coffee maker, iron, or even a picture on the stark white walls. We asked to change our room and then were put in one of the older suites on the 15th floor. While this room was attractive, we were greeted to jack hammers at 8 every morning. The staff were all very nice but the accommodations were lacking. lake shore drive",NEG,TRUE,Negative True,traveled chicago visit daughter life city suprise lovely older hotel major renovation gone lobby gust required walk basement get one elevator working second floor register first room recently renovatedstark white wall amenity guest enter room find way bed turn light room modern furniture includes bed close groud cold blck slab functioning desh metal chair fride coffee maker iron even picture stark white wall asked change room put one older suite 15th floor room attractive greeted jack hammer 8 every morning staff nice accommodation lacking lake shore drive
141,"The service was reasonably well...they seemed to have my reservation and checked me in fairly easily. The location was fine being that it was central to the city but I found out that it was also a tremendous drawback. It was interesting that the survey I took at the hotel in order to obtain internet access at the hotel asked what my most important quality was in a hotel visit and my response ""Quiet"". My visit was anything but, mainly attributed to the fact that they put me on the second floor, facing the street. I awoke at 4 a.m. and never did return to sleep because of the constant street noise and screeching of the L train that sounded as though it was just outside my window. Not the most conducive for a restful sleep prior to an important meeting. At check-out when I informed the hotel clerk of my dissatisfation he chuckled and said, "" Ah, city noise.""",NEG,TRUE,Negative True,service reasonably wellthey seemed reservation checked fairly easily location fine cent

### Replace th, st, and nd
- We can replace th, st, and nd with ""
- **IMPORTANT NOTE:** - Must replace 'st ' not 'st'.  
    - If you replace 'st' with no space, then it will the 'st in words like 'stay' and turn it into 'ay'

In [69]:
#Code Block 21

df_neg['cleaned_Reviews'] = df_neg.cleaned_Reviews.str.replace(' th ', '') 
df_neg['cleaned_Reviews'] = df_neg.cleaned_Reviews.str.replace(' st ', '') 
df_neg['cleaned_Reviews'] = df_neg.cleaned_Reviews.str.replace(' nd ', '') 

/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2608967778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['cleaned_Reviews'] = df_neg.cleaned_Reviews.str.replace(' th ', '')
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2608967778.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_neg['cleaned_Reviews'] = df_neg.cleaned_Reviews.str.replace(' st ', '')
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2608967778.py:5: SettingWithCopyWarning: 

### Search for 'th floor' now contains no values

In [72]:
#Code Block 22

df_neg[df_neg['cleaned_Reviews'].str.contains(" th floor")]

,Reviews,Type,Status,TypeStatus,cleaned_Reviews


### <h2 style="color:blue;">  Re-run Count Vectorization for Bigrams, Trigrams, and Quadgrams  </h2>

### CountVectorization for Bigrams

In [84]:
#Code Block 23

ngram_vect = CountVectorizer(ngram_range=(2,2), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_neg['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_2n = X_counts_df.sum()
comments_sum_2n = comments_sum_2n.sort_values(ascending=False)
comments_sum_2n.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Include Bigrams and Trigrams

In [86]:
#Code Block 24

ngram_vect = CountVectorizer(ngram_range=(2,3), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_neg['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_2_3n = X_counts_df.sum()
comments_sum_2_3n = comments_sum_2_3n.sort_values(ascending=False)
comments_sum_2_3n.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Why does front desk or customer service matter for negative reviews?

In [89]:
#Code Block 25

df_neg[df_neg['cleaned_Reviews'].str.contains("front desk")]

Reviews  \
0    My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one to normally write long email, I felt it necessary to bring some of the disappointing experiences from our recent syat (2/14 to 2/17) to your attention.Issues included: Our friends ordered chocolate covered strawberries to be delivered to our room upon arriving on 2/14 and we never got them. Our friends later (on 2/16) asked us if we had received the strawberries, and we responded no. They then brought this to the attention of the hotel and we were pleasantly surprised to have received a complimentary bottle of champagne. We were then disappointed again, upon checking out, however, when we were accidentally billed for the "complimentary" champagne - which was later subtracted from our bill. The in-room DVD player didn't work. It was approximately 11 PM on 2/15 when we called the front desk for help with trouble-shooting the issue. The person responded friendly that "Al, the engineer, will be calling you back shortly" to help. 10 minutes later there was a knock on the door, and it was Al. A great and friendly guy, but we were waiting for a call back, not the arrival of a service tech - keep in mind it was late and we were already in bed. When Al was not able to fix the DVD player, he graciously told us "order a movie on us" - thru the in-house movie selection. The problem: that didn't work either. Again, Al was awesome, and called the service provider, but they weren't able to fix it until after midnight. Finally, after midnight Al called us back, but by then the "movie watching" mood had sort of passed. I ended up watching the film "Hitman" alone as my wife had already gone to sleep.The thermostat in our room didn't work, regardless of the temperature we set it to (set to 65 degrees - actual temperature = 76 degrees). We had so many other unresolved issues that we didn't feel it necessary to voice any more. ...read more                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### Trigrams for Negative True

In [92]:
#Code Block 26

ngram_vect = CountVectorizer(ngram_range=(3,3), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_neg['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_3n = X_counts_df.sum()
comments_sum_3n = comments_sum_3n.sort_values(ascending=False)
comments_sum_3n.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Quadgrams for Negative True

In [95]:
#Code Block 27

ngram_vect = CountVectorizer(ngram_range=(4,4), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_neg['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_4n = X_counts_df.sum()
comments_sum_4n = comments_sum_4n.sort_values(ascending=False)
comments_sum_4n.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Create a DataFrame with the top 100 bigrams, trigrams and quadgrams

In [98]:
#Code Block 28

comments_sum_2n_100 = comments_sum_2n.head(100)
comments_sum_2n_100 = pd.DataFrame(comments_sum_2n_100).reset_index()
comments_sum_2n_100 = comments_sum_2n_100.rename(columns = {'index':'bigram', 0:'bi_freq'})
comments_sum_3n_100 = comments_sum_3n.head(100)
comments_sum_3n_100 = pd.DataFrame(comments_sum_3n_100).reset_index()
comments_sum_3n_100 = comments_sum_3n_100.rename(columns = {'index':'trigram', 0:'tri_freq'})
comments_sum_4n_100 = comments_sum_4n.head(100)
comments_sum_4n_100 = pd.DataFrame(comments_sum_4n_100).reset_index()
comments_sum_4n_100 = comments_sum_4n_100.rename(columns = {'index':'quadgram', 0:'quad_freq'})
neg_100 = pd.concat([comments_sum_2n_100, comments_sum_3n_100, comments_sum_4n_100], axis=1)
neg_100.head(20)

NameError: name 'comments_sum_2n' is not defined

### Add Vectoried table to Negative True Reviews

In [101]:
#Code Block 29

X_counts_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,28021,28022,28023,28024,28025,28026,28027,28028,28029,28030,28031,28032,28033,28034,28035,28036,28037,28038,28039,28040,28041,28042,28043,28044,28045,28046,28047,28048,28049,28050,28051,28052,28053,28054,28055,28056,28057,28058,28059,28060,28061,28062,28063,28064,28065,28066,28067,28068,28069,28070,28071,28072,28073,28074,28075,28076,28077,28078,28079,28080,28081,28082,28083,28084,28085,28086,28087,28088,28089,28090,28091,28092,28093,28094,28095,28096,28097,28098,28099,28100,28101,28102,28103,28104,28105,28106,28107,28108,28109,28110,28111,28112,28113,28114,28115,28116,28117,28118,28119,28120,28121,28122,28123,28124,28125,28126,28127,28128,28129,28130,28131,28132,28133,28134,28135,28136,28137,28138,28139,28140,28141,28142,28143,28144,28145,28146,28147,28148,28149,28150,28151,28152,28153,28154,28155,28156,28157,28158,28159,28160,28161,28162,28163,28164,28165,28166,28167,28168,28169,28170,28171,28172,28173,28174,28175,28176,28177,28178,28179,28180,28181,28182,28183,28184,28185,28186,28187,28188,28189,28190,28191,28192,28193,28194,28195,28196,28197,28198,28199,28200,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28211,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28230,28231,28232,28233,28234,28235,28236,28237,28238,28239,28240,28241,28242,28243,28244,28245,28246,28247,28248,28249,28250,28251,28252,28253,28254,28255,28256,28257,28258,28259,28260,28261,28262,28263,28264,28265,28266,28267,28268,28269,28270
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [103]:
#Code Block 30

pd.set_option('display.max_colwidth', 100)
neg_vector = pd.concat([df_neg, X_counts_df], axis=1)
neg_vector.head()

,Reviews,Type,Status,TypeStatus,cleaned_Reviews,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,28021,28022,28023,28024,28025,28026,28027,28028,28029,28030,28031,28032,28033,28034,28035,28036,28037,28038,28039,28040,28041,28042,28043,28044,28045,28046,28047,28048,28049,28050,28051,28052,28053,28054,28055,28056,28057,28058,28059,28060,28061,28062,28063,28064,28065,28066,28067,28068,28069,28070,28071,28072,28073,28074,28075,28076,28077,28078,28079,28080,28081,28082,28083,28084,28085,28086,28087,28088,28089,28090,28091,28092,28093,28094,28095,28096,28097,28098,28099,28100,28101,28102,28103,28104,28105,28106,28107,28108,28109,28110,28111,28112,28113,28114,28115,28116,28117,28118,28119,28120,28121,28122,28123,28124,28125,28126,28127,28128,28129,28130,28131,28132,28133,28134,28135,28136,28137,28138,28139,28140,28141,28142,28143,28144,28145,28146,28147,28148,28149,28150,28151,28152,28153,28154,28155,28156,28157,28158,28159,28160,28161,28162,28163,28164,28165,28166,28167,28168,28169,28170,28171,28172,28173,28174,28175,28176,28177,28178,28179,28180,28181,28182,28183,28184,28185,28186,28187,28188,28189,28190,28191,28192,28193,28194,28195,28196,28197,28198,28199,28200,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28211,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28230,28231,28232,28233,28234,28235,28236,28237,28238,28239,28240,28241,28242,28243,28244,28245,28246,28247,28248,28249,28250,28251,28252,28253,28254,28255,28256,28257,28258,28259,28260,28261,28262,28263,28264,28265,28266,28267,28268,28269,28270
0,"My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one...",NEG,TRUE,Negative True,wife spent long weekend hardrock chicago hotel although one normally write long email felt neces...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The historic feel of the hotel really had a strong influence on my choice to book this hotel. Un...,NEG,TRUE,Negative True,historic feel hotel really strong influence choice book hotel unfortunately ambiance hotel could...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [105]:
#Code Block 31

neg_vector.tail()

,Reviews,Type,Status,TypeStatus,cleaned_Reviews,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,...,28021,28022,28023,28024,28025,28026,28027,28028,28029,28030,28031,28032,28033,28034,28035,28036,28037,28038,28039,28040,28041,28042,28043,28044,28045,28046,28047,28048,28049,28050,28051,28052,28053,28054,28055,28056,28057,28058,28059,28060,28061,28062,28063,28064,28065,28066,28067,28068,28069,28070,28071,28072,28073,28074,28075,28076,28077,28078,28079,28080,28081,28082,28083,28084,28085,28086,28087,28088,28089,28090,28091,28092,28093,28094,28095,28096,28097,28098,28099,28100,28101,28102,28103,28104,28105,28106,28107,28108,28109,28110,28111,28112,28113,28114,28115,28116,28117,28118,28119,28120,28121,28122,28123,28124,28125,28126,28127,28128,28129,28130,28131,28132,28133,28134,28135,28136,28137,28138,28139,28140,28141,28142,28143,28144,28145,28146,28147,28148,28149,28150,28151,28152,28153,28154,28155,28156,28157,28158,28159,28160,28161,28162,28163,28164,28165,28166,28167,28168,28169,28170,28171,28172,28173,28174,28175,28176,28177,28178,28179,28180,28181,28182,28183,28184,28185,28186,28187,28188,28189,28190,28191,28192,28193,28194,28195,28196,28197,28198,28199,28200,28201,28202,28203,28204,28205,28206,28207,28208,28209,28210,28211,28212,28213,28214,28215,28216,28217,28218,28219,28220,28221,28222,28223,28224,28225,28226,28227,28228,28229,28230,28231,28232,28233,28234,28235,28236,28237,28238,28239,28240,28241,28242,28243,28244,28245,28246,28247,28248,28249,28250,28251,28252,28253,28254,28255,28256,28257,28258,28259,28260,28261,28262,28263,28264,28265,28266,28267,28268,28269,28270
395,The hotel was undergoing renovations so it was dirty and noisy. The entrance was really scary si...,NEG,TRUE,Negative True,hotel undergoing renovation dirty noisy entrance really scary since covered scaffolding making d...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
396,My son & I joined my husband on a work trip. We planned to swim while he was working. We overhea...,NEG,TRUE,Negative True,son joined husband work trip planned swim working overheard lady asking pool would open checking...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

<h2 style="color:blue;">  Clean and run count vectorization for positive true and compare to negative true reviews </h2>

In [109]:
#Code Block 32

display(df_pos.info())
df_pos.head()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 800 to 1199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Reviews     400 non-null    object
 1   Type        400 non-null    object
 2   Status      400 non-null    object
 3   TypeStatus  400 non-null    object
dtypes: object(4)
memory usage: 15.6+ KB


None

,Reviews,Type,Status,TypeStatus
800,The Hard Rock Hotel Chicago has become my favorite hotel. I've stayed there at least 5 times now...,POS,TRUE,Positive True
801,My wife and I had to stay downtown for event that we were attending at the Drake Hotel. I have s...,POS,TRUE,Positive True
802,"took a weekend trip with my wife. got a great rate with valet included, paid a couple extra buck...",POS,TRUE,Positive True
803,"We stayed here because of all the great reviews, and it's all true. We were highly impressed wit...",POS,TRUE,Positive True
804,Just returned from a week in Chicago with the family. This hotel was perfect for us. Location is...,POS,TRUE,Positive True


In [111]:
#Code Block 33

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([wordnet_lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return text

df_pos['cleaned_Reviews'] = df_pos['Reviews'].apply(lambda x: clean_text(x))
df_pos.head(2)

<>:5: SyntaxWarning: invalid escape sequence '\W'
<>:5: SyntaxWarning: invalid escape sequence '\W'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/3291887164.py:5: SyntaxWarning: invalid escape sequence '\W'
  tokens = re.split('\W+', text)
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/3291887164.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['cleaned_Reviews'] = df_pos['Reviews'].apply(lambda x: clean_text(x))


,Reviews,Type,Status,TypeStatus,cleaned_Reviews
800,The Hard Rock Hotel Chicago has become my favorite hotel. I've stayed there at least 5 times now...,POS,TRUE,Positive True,hard rock hotel chicago become favorite hotel ive stayed least 5 time never anything wonderful e...
801,My wife and I had to stay downtown for event that we were attending at the Drake Hotel. I have s...,POS,TRUE,Positive True,wife stay downtown event attending drake hotel stayed drake nice bit stuffy little outdated pric...


### Remove numbers and 'th ', 'st ', and 'nd '

In [114]:
#Code Block 34

df_pos['Reviews'] = df_pos['Reviews'].str.replace('\d+', '')
df_pos['cleaned_Reviews'] = df_pos.cleaned_Reviews.str.replace(' th ', '') 
df_pos['cleaned_Reviews'] = df_pos.cleaned_Reviews.str.replace(' st ', '') 
df_pos['cleaned_Reviews'] = df_pos.cleaned_Reviews.str.replace(' nd ', '') 


<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2580133451.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_pos['Reviews'] = df_pos['Reviews'].str.replace('\d+', '')
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2580133451.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pos['Reviews'] = df_pos['Reviews'].str.replace('\d+', '')
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2580133451.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

### CountVectorization for Bigrams for Positive True

In [145]:
#Code Block 35

ngram_vect = CountVectorizer(ngram_range=(2,2), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_pos['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_2p = X_counts_df.sum()
comments_sum_2p = comments_sum_2p.sort_values(ascending=False)
comments_sum_2p.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Trigrams for Positive True

In [120]:
#Code Block 36

ngram_vect = CountVectorizer(ngram_range=(3,3), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_pos['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_3p = X_counts_df.sum()
comments_sum_3p = comments_sum_3p.sort_values(ascending=False)
comments_sum_3p.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Quadgrams for Positive True

In [123]:
#Code Block 37

ngram_vect = CountVectorizer(ngram_range=(4,4), stop_words=stop_words)
X_counts = ngram_vect.fit_transform(df_pos['cleaned_Reviews'])


X_counts_df = pd.DataFrame(X_counts.toarray())
X_counts_df.columns = ngram_vect.get_feature_names()


comments_sum_4p = X_counts_df.sum()
comments_sum_4p = comments_sum_4p.sort_values(ascending=False)
comments_sum_4p.head(30)

InvalidParameterError: The 'stop_words' parameter of CountVectorizer must be a str among {'english'}, an instance of 'list' or None. Got {'hers', 'himself', 'and', 'd', 'same', 'wouldn', 'yourselves', 'it', 'does', 'doing', "you'll", 'here', 'only', 'shan', 'again', "shan't", 'm', 'is', 'few', 'their', 'couldn', 'from', 'by', "that'll", 'our', "aren't", 'yourself', 's', 'for', 'above', 'y', 'isn', 'this', 'she', 'that', 'an', 'needn', 'ours', 'about', 'not', "you'd", "she's", 'further', 'hasn', 're', 'weren', 'out', 'shouldn', "weren't", "isn't", 'of', 'then', 'once', "won't", 'those', "wouldn't", 'did', 'why', 'are', 'both', 'because', 'haven', 'i', 'being', 'my', 'very', 'him', 'themselves', 'more', "didn't", 'me', 'been', 'herself', 'against', 'at', 've', "couldn't", 'with', 'off', 'your', 'itself', "you're", 'when', "mustn't", 'we', 'until', 'through', 'most', 'won', 'other', 'no', 'has', 'do', 'ourselves', 'myself', "you've", 'will', 'how', 'on', "hadn't", 'below', 'just', 'his', 

### Create a DataFrame with the top 100 bigrams, trigrams and quadgrams for Positive True Reviews

In [126]:
#Code Block 38


comments_sum_2p_100 = comments_sum_2p.head(100)
comments_sum_2p_100 = pd.DataFrame(comments_sum_2p_100).reset_index()
comments_sum_2p_100 = comments_sum_2p_100.rename(columns = {'index':'bigram', 0:'bi_freq'})
comments_sum_3p_100 = comments_sum_3p.head(100)
comments_sum_3p_100 = pd.DataFrame(comments_sum_3p_100).reset_index()
comments_sum_3p_100 = comments_sum_3p_100.rename(columns = {'index':'trigram', 0:'tri_freq'})
comments_sum_4p_100 = comments_sum_4p.head(100)
comments_sum_4p_100 = pd.DataFrame(comments_sum_4p_100).reset_index()
comments_sum_4p_100 = comments_sum_4p_100.rename(columns = {'index':'quadgram', 0:'quad_freq'})
pos_100 = pd.concat([comments_sum_2p_100, comments_sum_3p_100, comments_sum_4p_100], axis=1)



NameError: name 'comments_sum_2p' is not defined

### Compare Negative True Reviews to Positive True Reviews

In [129]:
#Code Block 39

print("")
print('------------------------------------------------------------------------------------')
print('--------------------------   Top 20 Negative N-grams   -----------------------------')
display(neg_100.head(20))
print("")
print('------------------------------------------------------------------------------------')
print('--------------------------   Top 20 Positive N-grams   -----------------------------')
pos_100.head(20)


------------------------------------------------------------------------------------
--------------------------   Top 20 Negative N-grams   -----------------------------


NameError: name 'neg_100' is not defined

<h2 style="color:blue;"> Diving into common n-grams    </h2>

- Found the answer on Stack Overflow
    - https://stackoverflow.com/questions/45470373/extract-words-surrounding-a-word-and-inserting-results-in-a-dataframe-column
- Use the pandas.Series.str.extract 
    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html

In [132]:
#Code Block 40

pd.set_option('display.max_colwidth', 100)
df_neg.head(2)

,Reviews,Type,Status,TypeStatus,cleaned_Reviews
0,"My wife and I just spent a long weekend at the Hardrock Chicago hotel, and although I am not one...",NEG,TRUE,Negative True,wife spent long weekend hardrock chicago hotel although one normally write long email felt neces...
1,The historic feel of the hotel really had a strong influence on my choice to book this hotel. Un...,NEG,TRUE,Negative True,historic feel hotel really strong influence choice book hotel unfortunately ambiance hotel could...


In [134]:
#Code Block 41

pat = '(?P<before>(?:\w+\W+){,3})front desk\W+(?P<after>(?:\w+\W+){,3})'
neg_fd = df_neg.cleaned_Reviews.str.extract(pat, expand=True)

neg_fd.head(20)

<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/2994921704.py:3: SyntaxWarning: invalid escape sequence '\w'
  pat = '(?P<before>(?:\w+\W+){,3})front desk\W+(?P<after>(?:\w+\W+){,3})'


,before,after
0,pm 215 called,help troubleshooting issue
1,make rest shortcoming,staff rude doorman
2,immediately called hotel,explain predicament said
3,find reservation transferred,staff explain pet
4,NaN,NaN
5,checkout report problem,forget booked place
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [135]:
#Code Block 42

neg_fd = neg_fd.dropna(axis=0, how='any')
display(neg_fd.info())
neg_fd.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   before  82 non-null     object
 1   after   82 non-null     object
dtypes: object(2)
memory usage: 1.9+ KB


None

,before,after
0,pm 215 called,help troubleshooting issue
1,make rest shortcoming,staff rude doorman
2,immediately called hotel,explain predicament said
3,find reservation transferred,staff explain pet
5,checkout report problem,forget booked place
10,50 husband went,corrected met unfriendly
16,cooling whatever call,ask floor fan
19,packed bag checked,helpful kudos rocco
20,problem young woman,replied want tone
45,good start,isnt willing tell


In [137]:
#Code Block 43

pat = '(?P<before>(?:\w+\W+){,3})walking distance\W+(?P<after>(?:\w+\W+){,3})'
pos_fd = df_pos.cleaned_Reviews.str.extract(pat, expand=True)

pos_fd = pos_fd.dropna(axis=0, how='any')
display(neg_fd.info())
neg_fd.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   before  82 non-null     object
 1   after   82 non-null     object
dtypes: object(2)
memory usage: 1.9+ KB


<>:3: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\w'
/var/folders/ww/gvxz55x16sj2qt6_7hv99n0w0000gn/T/ipykernel_55803/1985215521.py:3: SyntaxWarning: invalid escape sequence '\w'
  pat = '(?P<before>(?:\w+\W+){,3})walking distance\W+(?P<after>(?:\w+\W+){,3})'


None

,before,after
0,pm 215 called,help troubleshooting issue
1,make rest shortcoming,staff rude doorman
2,immediately called hotel,explain predicament said
3,find reservation transferred,staff explain pet
5,checkout report problem,forget booked place
10,50 husband went,corrected met unfriendly
16,cooling whatever call,ask floor fan
19,packed bag checked,helpful kudos rocco
20,problem young woman,replied want tone
45,good start,isnt willing tell
